# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///Database_project_disaster.db')
df = pd.read_sql("table1",engine)
df.drop("original",axis = 1,inplace = True)
df.dropna(inplace = True)
category_names = df.columns

In [3]:
X = df["message"].values
y = df.drop(["message","genre","id"], axis = 1).values

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower().strip())
    
    tokens = word_tokenize(text)
    
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stopwords.words("english")]
   
    return tokens
    
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ("vect",CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [ ]:
y_pred = pipeline.predict(X_test)

In [8]:
def test_report(y_test,y_pred):
    
    # iterate through the columns of the labels    
    for i in range(y_test.shape[1]):
    
        y_test_col = y_test[:,i]
        y_pred_col = y_pred[:,i]
    
    # generate the classification report for the current column
    
        report = classification_report(y_test_col, y_pred_col)        

        print(f'Classification report for output {i+1}:')
        print(report)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
test_report(y_test,y_pred)

Classification report for output 1:
             precision    recall  f1-score   support

        0.0       0.35      0.06      0.10      2012
        1.0       0.76      0.97      0.85      6504
        2.0       0.25      0.02      0.03        55

avg / total       0.66      0.75      0.67      8571

Classification report for output 2:
             precision    recall  f1-score   support

        0.0       0.84      0.99      0.91      7111
        1.0       0.49      0.06      0.11      1460

avg / total       0.78      0.83      0.77      8571

Classification report for output 3:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      8530
        1.0       0.00      0.00      0.00        41

avg / total       0.99      1.00      0.99      8571

Classification report for output 4:
             precision    recall  f1-score   support

        0.0       0.59      0.82      0.69      5001
        1.0       0.46      0.21      0.29      357

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
pickle.dump(pipeline,open("classifierr.pkl","wb"))

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fc3028ea048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [17]:
parameters = {
        'clf__estimator__n_estimators': [50,75,100]
    }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [19]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

In [22]:
y_pred = cv.predict(X_test)

In [23]:
test_report(y_test,y_pred)

Classification report for output 1:
             precision    recall  f1-score   support

        0.0       0.37      0.06      0.11      2012
        1.0       0.76      0.97      0.85      6504
        2.0       0.40      0.04      0.07        55

avg / total       0.67      0.75      0.67      8571

Classification report for output 2:
             precision    recall  f1-score   support

        0.0       0.84      0.98      0.90      7111
        1.0       0.41      0.05      0.09      1460

avg / total       0.76      0.83      0.77      8571

Classification report for output 3:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      8530
        1.0       0.00      0.00      0.00        41

avg / total       0.99      1.00      0.99      8571

Classification report for output 4:
             precision    recall  f1-score   support

        0.0       0.59      0.82      0.69      5001
        1.0       0.46      0.21      0.29      357

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
acc = (y_pred == y_test).mean()
print(acc)

0.925543499397


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [27]:
pickle.dump(cv,open("classifier.pkl","wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.